In [1]:
import torch
import numpy as np
import pandas as pd

In [2]:
import pickle
with open('/itf-fi-ml/shared/users/ziyuzh/svm/data/scgpt/scgpt_data.pkl', 'rb') as f:
    data = pickle.load(f)

gene_ids = data['gene_ids']
gene_embeddings = data['gene_embeddings']


In [ ]:
with open('/itf-fi-ml/shared/users/ziyuzh/svm/data/scgpt/scgpt_ids.pkl', 'rb') as f:
    scgpt_id_map = pickle.load(f)

In [4]:
gene_embeddings = gene_embeddings.detach().cpu().numpy()

In [5]:
gene_embeddings.shape

(60697, 512)

In [10]:
import os
local_stringdb = '/itf-fi-ml/shared/users/ziyuzh/svm/data/stringdb/2023'

ppidf = pd.read_csv(os.path.join(local_stringdb,'9606.protein.info.v12.0.txt'), sep='\t', header=0, usecols=['#string_protein_id', 'preferred_name'])
ppidf['preferred_name'] = ppidf['preferred_name'].str.upper()
stringId2name = ppidf.set_index('#string_protein_id')['preferred_name'].to_dict()
name2stringId = ppidf.set_index('preferred_name')['#string_protein_id'].to_dict()
ppidf = pd.read_csv(os.path.join(local_stringdb,'9606.protein.aliases.v12.0.txt'), sep='\t', header=0, usecols=['#string_protein_id', 'alias']).drop_duplicates(['alias'], keep='first')
ppidf['alias'] = ppidf['alias'].str.upper()
aliases2stringId = ppidf.set_index('alias')['#string_protein_id'].to_dict()

def string_convert(gene):
    if gene in name2stringId.keys():
        return name2stringId[gene]
    elif gene in aliases2stringId.keys():
        return aliases2stringId[gene]
    else:
        return None

In [100]:
len(set(list(scgpt_id_map['gene2id'].keys()))&set(list(name2stringId.keys())))

18918

In [103]:
subid = []
features_pos = []
for id in scgpt_id_map['gene2id']:
    if id in list(name2stringId.keys()):
        subid.append(id)
        features_pos.append(scgpt_id_map['gene2id'][id])

In [104]:
len(subid),len(features_pos)

(18918, 18918)

In [106]:
# Create DataFrame
scgpt_df = pd.DataFrame(gene_embeddings[features_pos], index=subid)

# Rename columns to feature_0, feature_1, ...
scgpt_df.columns = [f'feature_{i}' for i in range(scgpt_df.shape[1])]

In [107]:
scgpt_df.head(3)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_502,feature_503,feature_504,feature_505,feature_506,feature_507,feature_508,feature_509,feature_510,feature_511
PYDC1,1.064177,-1.249982,0.629683,-1.538278,0.180076,-0.949641,0.403246,0.286524,-0.133856,-0.379318,...,-0.136490,-0.562780,-2.217903,0.499570,0.797949,-0.810311,0.364737,-1.109083,-0.263903,-1.437788
PYCR3,-0.500752,1.507778,1.805308,-1.261306,-0.908836,-0.752232,-1.694244,1.870296,-1.382904,-1.959757,...,1.047563,-0.528216,0.212154,-0.007878,0.301787,0.442021,0.306212,0.457046,-0.021947,-1.503676
BAG3,1.355876,-0.743320,1.583553,-0.888543,-0.563335,-1.299001,-0.281412,1.310264,1.921778,1.641888,...,-1.941576,0.952118,-1.364341,-1.445361,1.788207,-0.851026,1.855247,1.261890,0.205776,0.811407


In [108]:
scgpt_df['string_id'] = scgpt_df.index.map(name2stringId)

In [109]:
len(scgpt_df['string_id'].unique())

18918

In [110]:
scgpt_df.to_csv('/itf-fi-ml/shared/users/ziyuzh/svm/data/scgpt/scgpt_full.csv',index=False)